## ONNX to SNN-ONNX
- 기존 Keras로 작성한 DNN 모델을 ONNX로 변환
- ONNX로 변환된 모델을 SNN-ONNX로 변환 (활성화 함수를 - LIF 형태로 변환)

In [10]:
import os
import onnx

# 기존 ONNX 모델을 Load 하고 SNN-ONNX 로 변환
# 활성화 함수 LIF 형태로 변환

class convert_snnOnnx:
    def __init__(self):
        return

    def run(self, onnx_path, result_path, neuron_type):
        MODEL_SAVE_FOLDER_PATH = '../model/'

        # 경로에 폴더 미존재시 디렉토리 생성
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)
        
        sn_index = 1 # 활성화 함수 이름 라벨링 필요 변수
        
        # ONNX MODEL LOAD
        onnx_model = onnx.load(onnx_path)
        onnx_model.producer_name = 'onnx-snn'
        node_len = len(onnx_model.graph.node)
        
        # 각 ONNX node 들 접근.
        for index in range(node_len):
            # 각 op_type 들이 relu, sigmoid, tanh 등이면 neruon_type 으로 변경 (LIF)
            # LIF_1, LIF_2, LIF_3 ... 이렇게 변환해나감.
            op_type = onnx_model.graph.node[index].op_type.lower()
            
            # 활성화 함수가 relu, sigmoid, tanh 이면 -> LIF 로 변환
            if op_type == "relu" or op_type == "sigmoid" or op_type == "tanh":
                onnx_model.graph.node[index].op_type = neuron_type # 문자열
                onnx_model.graph.node[index].name = neuron_type + "_" + str(sn_index)
                sn_index = sn_index + 1
        onnx.save(onnx_model, result_path)

# keras to onnx (o)
# nengo_dl to onnx  #

if __name__ == "__main__":
    onnx_file_path = "../model/lenet-1.onnx"
    result_file_path = "../model/lenet-1_tanh_snn.onnx"
    cso = convert_snnOnnx() # 객체 생성
    cso.run(onnx_file_path, result_file_path, "LIF") # 활성화함수 LIF 로 변환
    print('변환완료')

변환완료


In [15]:
onnx.load(result_file_path).graph.node

[input: "conv1_input"
output: "adjusted_input1"
name: "Transpose4"
op_type: "Transpose"
attribute {
  name: "perm"
  ints: 0
  ints: 3
  ints: 1
  ints: 2
  type: INTS
}
doc_string: ""
domain: ""
, input: "adjusted_input1"
input: "conv1_4/kernel:0"
input: "conv1_4/bias:0"
output: "convolution_output1"
name: "conv1"
op_type: "Conv"
attribute {
  name: "auto_pad"
  s: "NOTSET"
  type: STRING
}
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 5
  ints: 5
  type: INTS
}
attribute {
  name: "pads"
  ints: 2
  ints: 2
  ints: 2
  ints: 2
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
domain: ""
, input: "convolution_output1"
output: "activation_3_1/Tanh:0"
name: "LIF_1"
op_type: "LIF"
doc_string: ""
domain: ""
, input: "activation_3_1/Tanh:0"
output: "activation_3_1/Tanh:0_pooling0"
name: "activation_3_1/Tanh:0_pooling"
op_type: "AveragePool"
attribut